In [33]:
import requests

from qdrant_client import QdrantClient
from qdrant_client.http import models

#### Qdrant

In [2]:
client = QdrantClient("http://localhost:6333")

In [5]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [10]:
documents_raw[0]["documents"][0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?'}

#### FastEmbed

In [26]:
import json

from fastembed import TextEmbedding

models = TextEmbedding.list_supported_models()

In [28]:
embedding_dim = 512

for model in models:
    if model["dim"] == embedding_dim:
        print(f"""model: {model["model"]}
            dim: {model["dim"]}
            size: {model["size_in_GB"]}
            desc: {model["description"]}
        """)

model: BAAI/bge-small-zh-v1.5
            dim: 512
            size: 0.09
            desc: Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.
        
model: Qdrant/clip-ViT-B-32-text
            dim: 512
            size: 0.25
            desc: Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year
        
model: jinaai/jina-embeddings-v2-small-en
            dim: 512
            size: 0.12
            desc: Text embeddings, Unimodal (text), English, 8192 input tokens truncation, Prefixes for queries/documents: not necessary, 2023 year.
        


In [30]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

Collection creation

In [34]:
collection_name = "zoomcamp-rag"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=embedding_dim,
        distance=models.Distance.COSINE
    )
)

True

Upserting points

In [37]:
points = []
id = 0

for course in documents_raw:
    for doc in course["documents"]:
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc["text"], model=model_handle),
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"]
            }
        )
        points.append(point)
        id += 1
        

In [39]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

---
Semantic search

In [61]:
def search(query, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        limit=limit,
        with_payload=True
    )

    return results
    

In [44]:
import random

course = random.choice(documents_raw)
doc = random.choice(course["documents"])
print(json.dumps(doc, indent=2))

{
  "text": "Problem description. How can we use Jupyter notebooks with the Pipenv environment?\nSolution: Refer to this stackoverflow question. Basically install jupyter and ipykernel using pipenv. And then register the kernel with `python -m ipykernel install --user --name=my-virtualenv-name` inside the Pipenv shell. If you are using Jupyter notebooks in VS Code, doing this will also add the virtual environment in the list of kernels.\nAdded by Ron Medina",
  "section": "Module 4: Deployment",
  "question": "Pipenv with Jupyter"
}


In [46]:
result = search(doc["question"])
result

QueryResponse(points=[ScoredPoint(id=899, version=1, score=0.88146776, payload={'text': 'Problem description. How can we use Jupyter notebooks with the Pipenv environment?\nSolution: Refer to this stackoverflow question. Basically install jupyter and ipykernel using pipenv. And then register the kernel with `python -m ipykernel install --user --name=my-virtualenv-name` inside the Pipenv shell. If you are using Jupyter notebooks in VS Code, doing this will also add the virtual environment in the list of kernels.\nAdded by Ron Medina', 'section': 'Module 4: Deployment', 'course': 'mlops-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [51]:
print(f"""question: {doc["question"]}\n""")
print(f"""retrieved_ans: {result.points[0].payload["text"]}\n""")
print(f""""original ans: {doc["text"]}""")

question: Pipenv with Jupyter

retrieved_ans: Problem description. How can we use Jupyter notebooks with the Pipenv environment?
Solution: Refer to this stackoverflow question. Basically install jupyter and ipykernel using pipenv. And then register the kernel with `python -m ipykernel install --user --name=my-virtualenv-name` inside the Pipenv shell. If you are using Jupyter notebooks in VS Code, doing this will also add the virtual environment in the list of kernels.
Added by Ron Medina

"original ans: Problem description. How can we use Jupyter notebooks with the Pipenv environment?
Solution: Refer to this stackoverflow question. Basically install jupyter and ipykernel using pipenv. And then register the kernel with `python -m ipykernel install --user --name=my-virtualenv-name` inside the Pipenv shell. If you are using Jupyter notebooks in VS Code, doing this will also add the virtual environment in the list of kernels.
Added by Ron Medina


In [62]:
print(search("How to run Kafka?").points[0].payload["text"])

For example, when running JsonConsumer.java, got:
Consuming form kafka started
RESULTS:::0
RESULTS:::0
RESULTS:::0
Or when running JsonProducer.java, got:
Exception in thread "main" java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.SaslAuthenticationException: Authentication failed
Solution:
Make sure in the scripts in src/main/java/org/example/ that you are running (e.g. JsonConsumer.java, JsonProducer.java), the StreamsConfig.BOOTSTRAP_SERVERS_CONFIG is the correct server url (e.g. europe-west3 from example vs europe-west2)
Make sure cluster key and secrets are updated in src/main/java/org/example/Secrets.java (KAFKA_CLUSTER_KEY and KAFKA_CLUSTER_SECRET)


---
Indexing on the payload fields

In [55]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

---
Search with filter

In [59]:
def search_in_course(query, course="mlops-zoomcamp", limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit,
        with_payload=True
    )

    return results
    

In [63]:
print(search_in_course("How to run Kafka?", "mlops-zoomcamp").points[0].payload["text"])

You have something running on the 5000 port. You need to stop it.
Answer: On terminal in mac .
Run ps -A | grep gunicorn
Look for the number process id which is the 1st number after running the command
kill 13580
where 13580  represents the process number.
Source
warrie.warrieus@gmail.com
Or by executing the following command it will kill all the processes using port 5000:
>> sudo fuser -k 5000/tcp
Answered by Vaibhav Khandelwal
Just execute in the command below in he command line to kill the running port
->> kill -9 $(ps -A | grep python | awk '{print $1}')
Answered by kamaldeen (kamaldeen32@gmail.com)
Change to different port (5001 in this case)
>> mlflow ui --backend-store-uri sqlite:///mlflow.db --port 5001
Answered by krishna (nellaikrishna@gmail.com)
